### Libraries

In [1]:
from torch.utils.data import DataLoader, random_split
from torch.optim import Adam, RMSprop

In [2]:
from data.preprocessing import *
from data.data_utils import *
from models.macro_architectures import *
from models.micro_architectures import *
from utils import *


# DATA

In [3]:
#format: YYYYMMDD
start_time = '20201231'
end_time = '20230803'
scrap_date = interval_time(start_time, end_time)

In [4]:
months = list(set([day[:6] for day in scrap_date]))
import_Dst(months)

In [5]:
l1_sample, l2_sample, dst, kp = automated_preprocessing(scrap_date, sep = False)

c:\Users\Innotronics\Desktop\SMFGF-SpaceApps\data\preprocessing.py:158: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  mg1 = pd.concat(mg1_list)


# L1 (raw) data

## Minute based

In [9]:
l1_sample

,proton_vx_gse,proton_vy_gse,proton_vz_gse,proton_vx_gsm,proton_vy_gsm,proton_vz_gsm,proton_speed,proton_density,proton_temperature,bt,bx_gse,by_gse,bz_gse,theta_gse,phi_gse,bx_gsm,by_gsm,bz_gsm,theta_gsm,phi_gsm
2020-12-31 00:00:00,-459.95712,20.771427,-37.828570,-459.95712,29.357143,-31.628570,462.02856,6.050000,154177.420,3.103543,-1.372186,-1.481739,-2.347711,-49.236874,212.29163,-1.372186,-0.863934,-2.638353,-58.362057,212.29163
2020-12-31 00:01:00,-458.98570,18.614286,-40.314290,-458.98570,27.914286,-34.542860,461.18573,5.697143,143959.140,3.121529,-1.600045,-1.255240,-2.361683,-49.195217,201.94005,-1.600045,-0.641299,-2.596520,-56.349064,201.94005
2020-12-31 00:02:00,-460.80002,19.028570,-37.328570,-460.80002,27.571428,-31.542858,462.82858,5.862857,153136.140,3.229239,-1.357512,-2.009158,-1.942433,-37.763630,224.33852,-1.357512,-1.475083,-2.373543,-47.985750,224.33852
2020-12-31 00:03:00,-458.98750,21.837500,-39.425000,-458.98750,30.775002,-32.900000,461.37500,5.867500,146527.750,2.966850,-1.500373,-1.187357,-1.967772,-42.944990,200.96104,-1.500373,-0.672283,-2.197755,-48.643024,200.96104
2020-12-31 00:04:00,-460.91428,21.199999,-39.257140,-460.91428,30.085714,-32.928570,463.17142,5.845714,141359.140,2.932062,-1.586736,-1.057594,-2.214242,-49.151546,197.24292,-1.586736,-0.486791,-2.405081,-55.295593,197.24292
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-08-03 23:55:00,-458.07272,8.363636,-20.463636,-458.07272,5.927273,-21.281816,459.57272,0.124545,2000.000,9.148619,-7.701201,4.927823,0.234573,1.467366,147.38512,-7.701201,4.921561,-0.341616,-2.142453,147.41812
2023-08-03 23:56:00,-457.90000,31.890000,-21.760000,-457.90000,29.130000,-25.340000,462.02002,0.403000,4756.600,9.117077,-7.746156,4.771574,0.456540,2.871917,148.36537,-7.746156,4.792239,-0.103969,-0.652903,148.25417
2023-08-03 23:57:00,-460.56366,1.745455,-13.645455,-460.56366,0.136364,-13.754546,460.77274,0.833636,14152.454,9.129887,-7.796285,4.708487,0.402483,2.526587,148.86864,-7.796285,4.723234,-0.151295,-0.951697,148.78772
2023-08-03 23:58:00,-463.80002,-14.516666,-4.116667,-463.80002,-14.891667,-2.391667,465.94998,0.733333,9470.083,9.107132,-7.686605,4.833078,0.528950,3.330908,147.84564,-7.686605,4.861768,-0.041320,-0.259435,147.69276


## Hour based

In [10]:
l1_sample_hour = l1_sample.resample('60min').mean()
l1_sample_hour

,proton_vx_gse,proton_vy_gse,proton_vz_gse,proton_vx_gsm,proton_vy_gsm,proton_vz_gsm,proton_speed,proton_density,proton_temperature,bt,bx_gse,by_gse,bz_gse,theta_gse,phi_gse,bx_gsm,by_gsm,bz_gsm,theta_gsm,phi_gsm
2020-12-31 00:00:00,-461.712648,23.564702,-41.086875,-461.712648,32.661459,-34.307589,464.346162,5.811333,146978.770000,3.196159,-1.654849,-1.847285,-1.694565,-33.542051,215.702141,-1.654849,-1.390457,-2.081981,-41.945192,215.702141
2020-12-31 01:00:00,-457.052138,17.513274,-27.782113,-457.052138,23.151429,-23.313929,458.318929,6.294155,197023.787333,2.760782,-0.405533,-2.105541,0.021299,-0.177205,258.416425,-0.405533,-2.051862,-0.433943,-11.797873,258.416425
2020-12-31 02:00:00,-453.349376,12.962054,-25.639137,-453.349376,17.454822,-22.815923,454.335716,6.812452,230582.338333,2.872992,0.120363,-2.265247,0.865786,20.457005,270.883119,0.120363,-2.386412,0.432359,10.488156,270.883119
2020-12-31 03:00:00,-457.754936,15.503958,-34.397917,-457.754936,20.284434,-31.818393,459.367022,6.874756,212633.102667,2.612051,-0.610070,-1.577324,-1.464512,-37.558328,237.178338,-0.610070,-1.357049,-1.681922,-44.019162,237.178338
2020-12-31 04:00:00,-441.350564,14.754673,-28.015595,-441.350564,17.365982,-26.479821,442.711253,6.524396,211812.393500,2.835215,-0.076185,-2.472715,0.753232,14.703117,268.512925,-0.076185,-2.524382,0.514577,9.325356,268.512925
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-08-03 19:00:00,-336.646294,2.560524,-40.894559,-336.646294,-4.469508,-40.811081,339.976270,3.297642,10603.785207,8.881810,-8.397860,-0.315262,2.761895,18.125610,182.149904,-8.397860,0.161601,2.780559,18.254308,178.913148
2023-08-03 20:00:00,-332.942454,-19.543290,-8.267044,-332.942454,-20.459794,-5.348601,335.023164,1.821652,6775.375603,9.278263,-8.971080,-0.909717,1.199496,7.538724,185.612646,-8.971080,-0.730964,1.308748,8.197190,184.480378
2023-08-03 21:00:00,-325.770046,5.254784,-28.044986,-325.770046,1.841405,-28.512537,328.366749,2.553373,7479.847265,9.491237,-9.264703,-0.280312,1.955153,11.896596,181.740103,-9.264703,-0.044151,1.975867,12.025461,180.281581
2023-08-03 22:00:00,-357.435846,15.239832,-27.647399,-357.435846,12.085765,-29.165588,359.615137,3.209151,65709.495525,9.317494,-9.089208,0.564847,1.426778,8.788789,176.384023,-9.089208,0.719256,1.356400,8.346369,175.425802


# L2 (cleaned) data

## Minute based

In [32]:
l2_sample

,proton_vx_gse,proton_vy_gse,proton_vz_gse,proton_vx_gsm,proton_vy_gsm,proton_vz_gsm,proton_speed,proton_density,proton_temperature,bt,bx_gse,by_gse,bz_gse,theta_gse,phi_gse,bx_gsm,by_gsm,bz_gsm,theta_gsm,phi_gsm
2020-12-31 00:00:00,-461.0,20.6,-37.7,-461.0,29.2,-31.5,463.0,6.32,156370.0,3.10,-1.37,-1.48,-2.35,-49.29,227.20,-1.37,-0.86,-2.64,-58.42,212.19
2020-12-31 00:01:00,-459.2,17.7,-39.2,-459.2,26.7,-33.7,461.2,6.03,145922.0,3.12,-1.59,-1.25,-2.36,-49.37,218.17,-1.59,-0.64,-2.60,-56.53,201.83
2020-12-31 00:02:00,-461.8,18.1,-39.2,-461.8,27.1,-33.6,463.8,6.12,152728.0,3.23,-1.40,-1.97,-2.02,-39.75,234.57,-1.40,-1.42,-2.44,-50.62,225.38
2020-12-31 00:03:00,-459.5,22.5,-38.1,-459.5,31.1,-31.5,461.6,6.33,144651.0,2.96,-1.60,-0.97,-2.16,-49.14,211.31,-1.60,-0.42,-2.33,-54.72,194.58
2020-12-31 00:04:00,-461.5,21.1,-38.5,-461.5,29.9,-32.2,463.6,6.04,141876.0,2.93,-1.58,-1.07,-2.21,-49.18,214.07,-1.58,-0.50,-2.41,-55.40,197.52
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-08-03 23:55:00,-458.3,1.8,-13.6,-458.3,0.2,-13.7,458.5,0.12,2000.0,9.15,-7.70,4.92,0.23,1.46,147.41,-7.70,4.92,-0.34,-2.14,147.44
2023-08-03 23:56:00,-457.8,1.7,-13.6,-457.8,0.1,-13.7,458.0,0.37,2000.0,9.12,-7.75,4.78,0.45,2.83,148.35,-7.75,4.80,-0.11,-0.69,148.24
2023-08-03 23:57:00,-460.4,1.7,-13.6,-460.4,0.1,-13.7,460.6,0.81,12657.0,9.13,-7.81,4.69,0.39,2.43,149.01,-7.81,4.70,-0.16,-1.03,148.94
2023-08-03 23:58:00,-463.8,1.8,-13.7,-463.8,0.2,-13.8,464.0,0.72,9024.0,9.11,-7.70,4.83,0.51,3.24,147.90,-7.70,4.86,-0.05,-0.35,147.75


## Hour based

In [12]:
l2_sample_hour = l2_sample.resample('60min').mean()
l2_sample_hour

,proton_vx_gse,proton_vy_gse,proton_vz_gse,proton_vx_gsm,proton_vy_gsm,proton_vz_gsm,proton_speed,proton_density,proton_temperature,bt,bx_gse,by_gse,bz_gse,theta_gse,phi_gse,bx_gsm,by_gsm,bz_gsm,theta_gsm,phi_gsm
2020-12-31 00:00:00,-462.145000,22.706667,-40.606667,-462.145000,31.723333,-34.036667,464.526667,6.024000,147974.900000,3.196333,-1.657667,-1.849500,-1.700000,-33.725000,225.520000,-1.657667,-1.390833,-2.088833,-42.233667,215.938167
2020-12-31 01:00:00,-457.590000,17.478333,-27.850000,-457.590000,23.135000,-23.376667,458.808333,6.346000,197041.233333,2.764000,-0.405833,-2.111667,0.024667,-0.183000,261.027000,-0.405833,-2.058167,-0.433333,-11.922167,259.814167
2020-12-31 02:00:00,-453.828333,13.290000,-25.736667,-453.828333,17.811667,-22.845000,454.766667,6.834000,233766.700000,2.872333,0.114333,-2.272667,0.877333,20.795500,269.079667,0.114333,-2.395333,0.441500,10.766000,271.607000
2020-12-31 03:00:00,-458.330000,15.601667,-34.370000,-458.330000,20.390000,-31.761667,459.913333,6.911000,213284.966667,2.611667,-0.609000,-1.573833,-1.474833,-38.339667,252.523333,-0.609000,-1.351500,-1.691833,-44.940667,239.751500
2020-12-31 04:00:00,-441.168333,14.753333,-27.376667,-441.168333,17.310000,-25.830000,442.366667,6.511333,211253.916667,2.838000,-0.072167,-2.477667,0.751000,14.600500,268.564667,-0.072167,-2.529833,0.510500,9.170167,268.554667
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-08-03 19:00:00,-336.648333,4.220000,-41.610000,-336.648333,-2.953333,-41.776667,339.760000,3.311167,10654.400000,8.882167,-8.398167,-0.316333,2.763167,18.135000,182.157833,-8.398167,0.161500,2.781667,18.264833,178.913500
2023-08-03 20:00:00,-331.586667,-20.721667,-8.181667,-331.586667,-21.613333,-5.080000,333.200000,1.798500,6084.233333,9.278833,-8.971667,-0.909500,1.197667,7.533833,185.614333,-8.971667,-0.730333,1.307333,8.195333,184.482500
2023-08-03 21:00:00,-325.763333,7.400000,-28.608333,-325.763333,3.903333,-29.333333,327.763333,2.565167,7487.116667,9.491667,-9.265500,-0.279167,1.955333,11.903167,181.734500,-9.265500,-0.042833,1.977167,12.030000,180.273833
2023-08-03 22:00:00,-357.271667,15.400000,-27.151667,-357.271667,12.301667,-28.686667,359.123333,3.185167,63931.000000,9.318167,-9.089667,0.566667,1.428333,8.796833,176.370833,-9.089667,0.721667,1.356833,8.353333,175.412000


# Dst data

In [13]:
dst

0       -3
1       -1
2       -1
3        0
4        0
        ..
22699    1
22700   -2
22701   -3
22702   -6
22703   -6
Name: Dst, Length: 22704, dtype: int64

# Kp data

In [14]:
kp

0       1+
1        1
2       0+
3        1
4       1-
        ..
7563     2
7564    2-
7565    1+
7566    0+
7567    0+
Name: Kp, Length: 7568, dtype: object

# Datasets
### Descriptions:
**hn_dl**: hour normal dataloader

**mn_dl**: minute normal dataloader

**hr_dl**: minute normal dataloader

**mr_dl**: minute normal dataloader



In [15]:
device = get_default_device()

In [16]:
sequence_length_hour = 10  #hour
sequence_length_minute = 600 #minute
pred_length = 9 #hours

In [49]:
##Normal
hour_Normal_dataset = NormalTrainingDataset(l1_sample_hour, dst, kp, sequence_length_hour, pred_length, hour = True)
minute_Normal_dataset = NormalTrainingDataset(l1_sample, dst, kp, sequence_length_minute, pred_length, hour = False)
##Refined(new method)
hour_Refined_dataset = RefinedTrainingDataset(l1_sample_hour, l2_sample_hour, dst,kp,sequence_length_hour, pred_length, hour = True)
minute_Refined_dataset = RefinedTrainingDataset(l1_sample, l2_sample, dst,kp,sequence_length_minute, pred_length, hour = False)

In [18]:
#Test:5% training: 95%

test_size = round(0.05*len(hour_Normal_dataset))

train_hn_ds, test_hn_ds = random_split(hour_Normal_dataset , [len(hour_Normal_dataset) - test_size, test_size])

batch_size = 32  #Change based on GPU capacity

train_hn_dl = DataLoader(train_hn_ds, batch_size, shuffle=True, num_workers=4, pin_memory=True)
train_hn_dl = DeviceDataLoader(train_hn_dl, device)
test_hn_dl = DataLoader(test_hn_ds, batch_size*2, num_workers=4, pin_memory=True)
test_hn_dl = DeviceDataLoader(test_hn_dl, device)

In [19]:
#Test:5% training: 95%

test_size = round(0.05*len(minute_Normal_dataset))

train_mn_ds, test_mn_ds = random_split(minute_Normal_dataset , [len(minute_Normal_dataset) - test_size, test_size])

batch_size = 32  #Change based on GPU capacity

train_mn_dl = DataLoader(train_mn_ds, batch_size, shuffle=True, num_workers=4, pin_memory=True)
train_mn_dl = DeviceDataLoader(train_mn_dl, device)
test_mn_dl = DataLoader(test_mn_ds, batch_size*2, num_workers=4, pin_memory=True)
test_mn_dl = DeviceDataLoader(test_mn_dl, device)


In [20]:
#Test:5% training: 95%

test_size = round(0.05*len(hour_Refined_dataset))

train_hr_ds, test_hr_ds = random_split(hour_Refined_dataset , [len(hour_Refined_dataset) - test_size, test_size])

batch_size = 32  #Change based on GPU capacity

train_hr_dl = DataLoader(train_hr_ds, batch_size, shuffle=True, num_workers=4, pin_memory=True)
train_hr_dl = DeviceDataLoader(train_hr_dl, device)
test_hr_dl = DataLoader(test_hr_ds, batch_size*2, num_workers=4, pin_memory=True)
test_hr_dl = DeviceDataLoader(test_hr_dl, device)

In [21]:
#Test:5% training: 95%

test_size = round(0.05*len(minute_Refined_dataset))

train_mr_ds, test_mr_ds = random_split(minute_Refined_dataset , [len(minute_Refined_dataset) - test_size, test_size])

batch_size = 32  #Change based on GPU capacity

train_mr_dl = DataLoader(train_mr_ds, batch_size, shuffle=True, num_workers=4, pin_memory=True)
train_mr_dl = DeviceDataLoader(train_mr_dl, device)
test_mr_dl = DataLoader(test_mr_ds, batch_size*2, num_workers=4, pin_memory=True)
test_mr_dl = DeviceDataLoader(test_mr_dl, device)

# Models

## Refined models with attention

### 1d Convolutional encoder

In [23]:
## Architecture
hidden_size = 10
input_size = 20
##nn architecture
architecture = (10,10,10)

In [24]:
#encoders
cnn_encoder_min = Simple1DCNN(architecture, sequence_length_minute, hidden_size)
cnn_encoder_hour = Simple1DCNN(architecture, sequence_length_hour, hidden_size)
#fc layer
cnn_fc_dst_min = DeepNeuralNetwork(hidden_size, pred_length, *architecture)
cnn_fc_kp_min = DeepNeuralNetwork(hidden_size, hour_to_3_hour(pred_length), *architecture)

cnn_fc_dst_hour = DeepNeuralNetwork(hidden_size, pred_length, *architecture)
cnn_fc_kp_hour = DeepNeuralNetwork(hidden_size, hour_to_3_hour(pred_length), *architecture)

In [25]:
RefinedCNN_min = to_device(RefinedArchitecture(cnn_encoder_min, cnn_fc_dst_min, cnn_fc_kp_min), device)
RefinedCNN_hour = to_device(RefinedArchitecture(cnn_encoder_hour, cnn_fc_dst_hour, cnn_fc_kp_hour), device)

In [26]:
##hyperparameters
epochs = 20
max_lr = 5e-4
weigth_decay = 1e-6
grad_clip = 1e-1
opt_func = Adam
#opt_func = RMSprop

In [27]:
RefinedCNN_min_history = []

In [33]:
RefinedCNN_min_history += RefinedCNN_min.fit(epochs, max_lr, train_mr_dl, test_mr_dl, weigth_decay, grad_clip, opt_func)

KeyboardInterrupt: 

In [29]:
RefinedCNN_hour_history = []

In [34]:
RefinedCNN_hour_history += RefinedCNN_hour.fit(epochs, max_lr, train_hr_dl, test_hr_dl, weigth_decay, grad_clip, opt_func)

Epoch [0]
	last_lr: 0.00005
	train_overall_loss: 1.7226
	train_main_loss: 1.7226
	train_output_loss: 0.0001
	train_encoder_loss: 0.0010
	val_overall_loss: 1.7158
	val_main_loss: 1.7158
	val_output_loss: 0.0002
	val_encoder_loss: 0.0015
Epoch [1]
	last_lr: 0.00014
	train_overall_loss: 1.6239
	train_main_loss: 1.6239
	train_output_loss: 0.0003
	train_encoder_loss: 0.0016
	val_overall_loss: 1.6137
	val_main_loss: 1.6137
	val_output_loss: 0.0004
	val_encoder_loss: 0.0021


### Bidirectional Deep LSTM encoder

In [35]:
## Architecture
hidden_size = 10
input_size = 20
##nn architecture
architecture = ()

In [43]:
#encoders
##Bidirectional minute based models
deep_lstm_encoder_min_forward = DeepLSTM(hidden_size, input_size, architecture, attention = True).to(device)
deep_lstm_encoder_min_backward = DeepLSTM(hidden_size, input_size, architecture, attention = True).to(device)
bidirectional_deeplstm_encoder_min = BidirectionalRNN(deep_lstm_encoder_min_forward, deep_lstm_encoder_min_backward).to(device)
##Bidirectional hour based models
deep_lstm_encoder_hour_forward = DeepLSTM(hidden_size, input_size, architecture, attention = True).to(device)
deep_lstm_encoder_hour_backward = DeepLSTM(hidden_size, input_size, architecture, attention = True).to(device)
bidirectional_deeplstm_encoder_hour = BidirectionalRNN(deep_lstm_encoder_hour_forward, deep_lstm_encoder_hour_backward).to(device)
#fc layer
deep_lstm_fc_dst_min = DeepNeuralNetwork(2*hidden_size, pred_length, *architecture).to(device) #the multiplying factor because concatenating hidden_states on bidirectional arch
deep_lstm_fc_kp_min = DeepNeuralNetwork(2*hidden_size, hour_to_3_hour(pred_length), *architecture).to(device)

deep_lstm_fc_dst_hour = DeepNeuralNetwork(2*hidden_size, pred_length, *architecture).to(device)
deep_lstm_fc_kp_hour = DeepNeuralNetwork(2*hidden_size, hour_to_3_hour(pred_length), *architecture).to(device)

In [44]:
RefinedLSTM_min = to_device(RefinedArchitecture(bidirectional_deeplstm_encoder_min, deep_lstm_fc_dst_min, deep_lstm_fc_kp_min), device)
RefinedLSTM_hour = to_device(RefinedArchitecture(bidirectional_deeplstm_encoder_hour, deep_lstm_fc_dst_hour, deep_lstm_fc_kp_hour), device)

In [45]:
##hyperparameters
epochs = 10
max_lr = 1e-2
weigth_decay = 1e-3
grad_clip = 1e-1
opt_func = Adam
#opt_func = RMSprop

In [46]:
RefinedLSTM_min_history = []

In [47]:
RefinedLSTM_min_history += RefinedLSTM_min.fit(epochs, max_lr, train_mr_dl, test_mr_dl, weigth_decay, grad_clip, opt_func)

RuntimeError: Expected all tensors to be on the same device, but found at least two devices, cuda:0 and cpu! (when checking argument for argument mat1 in method wrapper_CUDA_addmm)

In [ ]:
RefinedLSTM_hour_history = []

In [ ]:
RefinedLSTM_hour_history += RefinedLSTM_hour.fit(epochs, max_lr, train_hr_dl, test_hr_dl, weigth_decay, grad_clip, opt_func)

Epoch [0]
	last_lr: 0.00004
	train_overall_loss: 1.3423
	train_main_loss: 1.3423
	train_output_loss: 0.0002
	train_encoder_loss: 0.0000
	val_overall_loss: 0.9733
	val_main_loss: 0.9733
	val_output_loss: 0.0002
	val_encoder_loss: 0.0000
Epoch [1]
	last_lr: 0.00005
	train_overall_loss: 0.9529
	train_main_loss: 0.9529
	train_output_loss: 0.0005
	train_encoder_loss: 0.0001
	val_overall_loss: 0.8512
	val_main_loss: 0.8512
	val_output_loss: 0.0004
	val_encoder_loss: 0.0001
Epoch [2]
	last_lr: 0.00006
	train_overall_loss: 0.8301
	train_main_loss: 0.8301
	train_output_loss: 0.0008
	train_encoder_loss: 0.0001
	val_overall_loss: 0.7807
	val_main_loss: 0.7807
	val_output_loss: 0.0004
	val_encoder_loss: 0.0001
Epoch [3]
	last_lr: 0.00008
	train_overall_loss: 0.7586
	train_main_loss: 0.7586
	train_output_loss: 0.0007
	train_encoder_loss: 0.0001
	val_overall_loss: 0.7456
	val_main_loss: 0.7456
	val_output_loss: 0.0005
	val_encoder_loss: 0.0001
Epoch [4]
	last_lr: 0.00010
	train_overall_loss: 0.7159


KeyboardInterrupt: 

### Bidirectional Deep GRU encoder

In [ ]:
## Architecture
hidden_size = 10
input_size = 20
##nn architecture
architecture = ()

In [ ]:
#encoders
##Bidirectional minute based models
deep_gru_encoder_min_forward= DeepGRU(hidden_size, input_size, architecture)
deep_gru_encoder_min_backward= DeepGRU(hidden_size, input_size, architecture)
bidirectional_deepgru_encoder_min = BidirectionalRNN(deep_gru_encoder_min_forward, deep_gru_encoder_min_backward)
##Bidirectional hour based models
deep_gru_encoder_hour_forward = DeepGRU(hidden_size, input_size, architecture)
deep_gru_encoder_hour_backward = DeepGRU(hidden_size, input_size, architecture)
bidirectional_deepgru_encoder_hour = BidirectionalRNN(deep_gru_encoder_hour_forward, deep_gru_encoder_hour_backward)
#fc layer
deep_gru_fc_dst_min = DeepNeuralNetwork(2*hidden_size, pred_length, *architecture)
deep_gru_fc_kp_min = DeepNeuralNetwork(2*hidden_size, hour_to_3_hour(pred_length), *architecture)

deep_gru_fc_dst_hour = DeepNeuralNetwork(2*hidden_size, pred_length, *architecture)
deep_gru_fc_kp_hour = DeepNeuralNetwork(2*hidden_size, hour_to_3_hour(pred_length), *architecture)

In [ ]:
RefinedGRU_min = to_device(RefinedArchitecture(bidirectional_deepgru_encoder_min, deep_gru_fc_dst_min, deep_gru_fc_kp_min), device)
RefinedGRU_hour = to_device(RefinedArchitecture(bidirectional_deepgru_encoder_hour, deep_gru_fc_dst_hour, deep_gru_fc_kp_hour), device)

In [ ]:
##hyperparameters
epochs = 100
max_lr = 1e-2
weigth_decay = 1e-6
grad_clip = 5e-1
opt_func = Adam
#opt_func = RMSprop

In [ ]:
RefinedGRU_min_history = []

In [ ]:
RefinedGRU_min_history += RefinedGRU_min.fit(epochs, max_lr, train_mr_dl, test_mr_dl, weigth_decay, grad_clip, opt_func)

In [ ]:
RefinedGRU_hour_history = []

In [31]:
RefinedGRU_hour_history += RefinedGRU_hour.fit(epochs, max_lr, train_hr_dl, test_hr_dl, weigth_decay, grad_clip, opt_func)

NameError: name 'RefinedGRU_hour_history' is not defined

### Bidirectional Deep Vanilla RNN encoder

In [ ]:
## Architecture
hidden_size = 10
input_size = 20
##nn architecture
architecture = ()

In [ ]:
#encoders
##Bidirectional minute encoders
deep_rnn_encoder_min_forward = DeepVanillaRNN(hidden_size, input_size, architecture)
deep_rnn_encoder_min_backward = DeepVanillaRNN(hidden_size, input_size, architecture)
bidirectional_deeprnn_encoder_min = BidirectionalRNN(deep_rnn_encoder_min_forward,deep_rnn_encoder_min_backward)
##Bidirectional hour encoders
deep_rnn_encoder_hour_forward = DeepVanillaRNN(hidden_size, input_size, architecture)
deep_rnn_encoder_hour_backward = DeepVanillaRNN(hidden_size, input_size, architecture)
bidirectional_deeprnn_encoder_hour = BidirectionalRNN(deep_rnn_encoder_hour_forward,deep_rnn_encoder_hour_backward)
#fc layer
deep_rnn_fc_dst_min = DeepNeuralNetwork(2*hidden_size, pred_length, *architecture)
deep_rnn_fc_kp_min = DeepNeuralNetwork(2*hidden_size, hour_to_3_hour(pred_length), *architecture)

deep_rnn_fc_dst_hour = DeepNeuralNetwork(2*hidden_size, pred_length, *architecture)
deep_rnn_fc_kp_hour = DeepNeuralNetwork(2*hidden_size, hour_to_3_hour(pred_length), *architecture)

In [ ]:
RefinedVanillaRNN_min = to_device(RefinedArchitecture(bidirectional_deeprnn_encoder_min, deep_rnn_fc_dst_min, deep_rnn_fc_kp_min), device)
RefinedVanillaRNN_hour = to_device(RefinedArchitecture(bidirectional_deeprnn_encoder_hour, deep_rnn_fc_dst_hour, deep_rnn_fc_kp_hour), device)

In [ ]:
##hyperparameters
epochs = 100
max_lr = 1e-3
weigth_decay = 1e-4
grad_clip = 1e-2
#opt_func = Adam
opt_func = RMSprop

In [ ]:
RefinedVanillaRNN_min_history = []

In [ ]:
RefinedVanillaRNN_min_history += RefinedVanillaRNN_min.fit(epochs, max_lr, train_mr_dl, test_mr_dl, weigth_decay, grad_clip, opt_func)

In [ ]:
RefinedVanillaRNN_hour_history = []

In [ ]:
RefinedVanillaRNN_hour_history += RefinedVanillaRNN_hour.fit(epochs, max_lr, train_hr_dl, test_hr_dl, weigth_decay, grad_clip, opt_func)

Epoch [0]
	last_lr: 0.00004
	train_overall_loss: 1.3494
	train_main_loss: 1.3494
	train_output_loss: 0.0001
	train_encoder_loss: 0.0001
	val_overall_loss: 1.2106
	val_main_loss: 1.2106
	val_output_loss: 0.0002
	val_encoder_loss: 0.0001
Epoch [1]
	last_lr: 0.00005
	train_overall_loss: 1.0410
	train_main_loss: 1.0410
	train_output_loss: 0.0003
	train_encoder_loss: 0.0001
	val_overall_loss: 1.0773
	val_main_loss: 1.0773
	val_output_loss: 0.0004
	val_encoder_loss: 0.0001


KeyboardInterrupt: 

### Bidirectional non deep architectures

#### LSTM

In [ ]:
non_deep_lstm_forward_min = LSTM(input_size, hidden_size)
non_deep_lstm_backward_min = LSTM(input_size, hidden_size)
bidirectional_lstm_encoder_min = BidirectionalRNNWithAttention(non_deep_lstm_forward_min, non_deep_lstm_backward_min)

non_deep_lstm_forward_hour = LSTM(input_size, hidden_size)
non_deep_lstm_backward_hour = LSTM(input_size, hidden_size)
bidirectional_lstm_encoder_hour = BidirectionalRNNWithAttention(non_deep_lstm_forward_hour, non_deep_lstm_backward_hour)
#fc layer
non_deep_lstm_fc_dst_min = DeepNeuralNetwork(2*hidden_size, pred_length, *architecture)
non_deep_lstm_fc_kp_min = DeepNeuralNetwork(2*hidden_size, hour_to_3_hour(pred_length), *architecture)

non_deep_lstm_fc_dst_hour = DeepNeuralNetwork(2*hidden_size, pred_length, *architecture)
non_deep_lstm_fc_kp_hour = DeepNeuralNetwork(2*hidden_size, hour_to_3_hour(pred_length), *architecture)

In [ ]:
RefinedNonDeepLSTM_min = to_device(RefinedArchitecture(bidirectional_lstm_encoder_min, non_deep_lstm_fc_dst_min, non_deep_lstm_fc_kp_min), device)
RefinedNonDeepLSTM_hour = to_device(RefinedArchitecture(bidirectional_lstm_encoder_hour, non_deep_lstm_fc_dst_hour, non_deep_lstm_fc_kp_hour), device)

In [ ]:
RefinedNonDeepLSTM_min_history = []

In [ ]:
RefinedNonDeepLSTM_min_history += RefinedNonDeepLSTM_min.fit(epochs, max_lr, train_mr_dl, test_mr_dl, weigth_decay, grad_clip, opt_func)

In [ ]:
RefinedNonDeepLSTM_hour_history = []

In [ ]:
RefinedNonDeepLSTM_hour_history += RefinedNonDeepLSTM_hour.fit(epochs, max_lr, train_hr_dl, test_hr_dl, weigth_decay, grad_clip, opt_func)

Epoch [0]
	last_lr: 0.00004
	train_overall_loss: 1.3338
	train_main_loss: 1.3338
	train_output_loss: 0.0002
	train_encoder_loss: 0.0001
	val_overall_loss: 1.1319
	val_main_loss: 1.1319
	val_output_loss: 0.0004
	val_encoder_loss: 0.0001
Epoch [1]
	last_lr: 0.00005
	train_overall_loss: 0.9605
	train_main_loss: 0.9605
	train_output_loss: 0.0005
	train_encoder_loss: 0.0001
	val_overall_loss: 0.9760
	val_main_loss: 0.9760
	val_output_loss: 0.0005
	val_encoder_loss: 0.0001
Epoch [2]
	last_lr: 0.00006
	train_overall_loss: 0.8491
	train_main_loss: 0.8491
	train_output_loss: 0.0008
	train_encoder_loss: 0.0002
	val_overall_loss: 0.8663
	val_main_loss: 0.8663
	val_output_loss: 0.0005
	val_encoder_loss: 0.0001
Epoch [3]
	last_lr: 0.00008
	train_overall_loss: 0.7699
	train_main_loss: 0.7698
	train_output_loss: 0.0008
	train_encoder_loss: 0.0002
	val_overall_loss: 0.7792
	val_main_loss: 0.7792
	val_output_loss: 0.0006
	val_encoder_loss: 0.0002
Epoch [4]
	last_lr: 0.00010
	train_overall_loss: 0.7140


KeyboardInterrupt: 

#### GRU

In [ ]:
non_deep_gru_forward_min = GRU(input_size, hidden_size)
non_deep_gru_backward_min = GRU(input_size, hidden_size)
bidirectional_gru_encoder_min = BidirectionalRNNWithAttention(non_deep_gru_forward_min, non_deep_gru_backward_min)

non_deep_gru_forward_hour = GRU(input_size, hidden_size)
non_deep_gru_backward_hour = GRU(input_size, hidden_size)
bidirectional_gru_encoder_hour = BidirectionalRNNWithAttention(non_deep_gru_forward_hour, non_deep_gru_backward_hour)
#fc layer
non_deep_gru_fc_dst_min = DeepNeuralNetwork(2*hidden_size, pred_length, *architecture)
non_deep_gru_fc_kp_min = DeepNeuralNetwork(2*hidden_size, hour_to_3_hour(pred_length), *architecture)

non_deep_gru_fc_dst_hour = DeepNeuralNetwork(2*hidden_size, pred_length, *architecture)
non_deep_gru_fc_kp_hour = DeepNeuralNetwork(2*hidden_size, hour_to_3_hour(pred_length), *architecture)


In [ ]:
RefinedNonDeepGRU_min = to_device(RefinedArchitecture(bidirectional_gru_encoder_min, non_deep_gru_fc_dst_min, non_deep_gru_fc_kp_min), device)
RefinedNonDeepGRU_hour = to_device(RefinedArchitecture(bidirectional_gru_encoder_hour, non_deep_gru_fc_dst_hour, non_deep_gru_fc_kp_hour), device)

In [ ]:
RefinedNonDeepGRU_min_history = []

In [ ]:
RefinedNonDeepGRU_min_history += RefinedNonDeepGRU_min.fit(epochs, max_lr, train_mr_dl, test_mr_dl, weigth_decay, grad_clip, opt_func)

In [ ]:
RefinedNonDeepGRU_hour_history = []

In [ ]:
RefinedNonDeepGRU_hour_history += RefinedNonDeepGRU_hour.fit(epochs, max_lr, train_hr_dl, test_hr_dl, weigth_decay, grad_clip, opt_func)

Epoch [0]
	last_lr: 0.00004
	train_overall_loss: 1.3516
	train_main_loss: 1.3516
	train_output_loss: 0.0002
	train_encoder_loss: 0.0001
	val_overall_loss: 1.1860
	val_main_loss: 1.1860
	val_output_loss: 0.0003
	val_encoder_loss: 0.0001
Epoch [1]
	last_lr: 0.00005
	train_overall_loss: 0.9823
	train_main_loss: 0.9823
	train_output_loss: 0.0003
	train_encoder_loss: 0.0001
	val_overall_loss: 1.0192
	val_main_loss: 1.0192
	val_output_loss: 0.0005
	val_encoder_loss: 0.0001
Epoch [2]
	last_lr: 0.00006
	train_overall_loss: 0.8637
	train_main_loss: 0.8637
	train_output_loss: 0.0005
	train_encoder_loss: 0.0001
	val_overall_loss: 0.9084
	val_main_loss: 0.9084
	val_output_loss: 0.0005
	val_encoder_loss: 0.0001
Epoch [3]
	last_lr: 0.00008
	train_overall_loss: 0.7905
	train_main_loss: 0.7905
	train_output_loss: 0.0005
	train_encoder_loss: 0.0001
	val_overall_loss: 0.8263
	val_main_loss: 0.8263
	val_output_loss: 0.0005
	val_encoder_loss: 0.0001
Epoch [4]
	last_lr: 0.00010
	train_overall_loss: 0.7357


KeyboardInterrupt: 

#### RNN

In [ ]:
non_deep_rnn_forward_min = VanillaRNN(input_size, hidden_size)
non_deep_rnn_backward_min = VanillaRNN(input_size, hidden_size)
bidirectional_rnn_encoder_min = BidirectionalRNNWithAttention(non_deep_rnn_forward_min, non_deep_rnn_backward_min)

non_deep_rnn_forward_hour = VanillaRNN(input_size, hidden_size)
non_deep_rnn_backward_hour = VanillaRNN(input_size, hidden_size)
bidirectional_rnn_encoder_hour = BidirectionalRNNWithAttention(non_deep_rnn_forward_hour, non_deep_rnn_backward_hour)
#fc layer
non_deep_rnn_fc_dst_min = DeepNeuralNetwork(2*hidden_size, pred_length, *architecture)
non_deep_rnn_fc_kp_min = DeepNeuralNetwork(2*hidden_size, hour_to_3_hour(pred_length), *architecture)

non_deep_rnn_fc_dst_hour = DeepNeuralNetwork(2*hidden_size, pred_length, *architecture)
non_deep_rnn_fc_kp_hour = DeepNeuralNetwork(2*hidden_size, hour_to_3_hour(pred_length), *architecture)


In [ ]:
RefinedNonDeepVanillaRNN_min = to_device(RefinedArchitecture(bidirectional_rnn_encoder_min, non_deep_rnn_fc_dst_min, non_deep_rnn_fc_kp_min), device)
RefinedNonDeepVanillaRNN_hour = to_device(RefinedArchitecture(bidirectional_rnn_encoder_hour, non_deep_rnn_fc_dst_hour, non_deep_rnn_fc_kp_hour), device)

In [ ]:
RefinedNonDeepVanillaRNN_min_history = []

In [ ]:
RefinedNonDeepVanillaRNN_min_history += RefinedNonDeepVanillaRNN_min.fit(epochs, max_lr, train_mr_dl, test_mr_dl, weigth_decay, grad_clip, opt_func)

In [ ]:
RefinedNonDeepVanillaRNN_hour_history = []

In [ ]:
RefinedNonDeepVanillaRNN_hour_history += RefinedNonDeepVanillaRNN_hour.fit(epochs, max_lr, train_hr_dl, test_hr_dl, weigth_decay, grad_clip, opt_func)

Epoch [0]
	last_lr: 0.00004
	train_overall_loss: 1.2946
	train_main_loss: 1.2946
	train_output_loss: 0.0002
	train_encoder_loss: 0.0002
	val_overall_loss: 1.1871
	val_main_loss: 1.1871
	val_output_loss: 0.0003
	val_encoder_loss: 0.0002
Epoch [1]
	last_lr: 0.00005
	train_overall_loss: 1.0257
	train_main_loss: 1.0257
	train_output_loss: 0.0003
	train_encoder_loss: 0.0002
	val_overall_loss: 1.0831
	val_main_loss: 1.0830
	val_output_loss: 0.0005
	val_encoder_loss: 0.0002
Epoch [2]
	last_lr: 0.00006
	train_overall_loss: 0.9554
	train_main_loss: 0.9554
	train_output_loss: 0.0005
	train_encoder_loss: 0.0002
	val_overall_loss: 1.0201
	val_main_loss: 1.0201
	val_output_loss: 0.0006
	val_encoder_loss: 0.0002


KeyboardInterrupt: 

## Normal models with attention

### 1d Convolutional encoder

In [ ]:
## Architecture
hidden_size = 10
input_size = 20
##nn architecture
architecture = ()

In [ ]:
#encoders
cnn_encoder_min = Simple1DCNN(architecture, input_size, hidden_size)
cnn_encoder_hour = Simple1DCNN(architecture, input_size, hidden_size)
#fc layer
cnn_fc_dst_min = DeepNeuralNetwork(hidden_size, pred_length, *architecture)
cnn_fc_kp_min = DeepNeuralNetwork(hidden_size, hour_to_3_hour(pred_length), *architecture)

cnn_fc_dst_hour = DeepNeuralNetwork(hidden_size, pred_length, *architecture)
cnn_fc_kp_hour = DeepNeuralNetwork(hidden_size, hour_to_3_hour(pred_length), *architecture)

In [ ]:
NormalCNN_min = to_device(NormalArchitecture(cnn_encoder_min, cnn_fc_dst_min, cnn_fc_kp_min), device)
NormalCNN_hour = to_device(NormalArchitecture(cnn_encoder_hour, cnn_fc_dst_hour, cnn_fc_kp_hour), device)

In [ ]:
##hyperparameters
epochs = 100
max_lr = 1e-3
weigth_decay = 1e-4
grad_clip = 1e-2
#opt_func = Adam
opt_func = RMSprop

In [ ]:
NormalCNN_min_history = []

In [ ]:
NormalCNN_min_history += NormalCNN_min.fit(epochs, max_lr, train_mn_dl, test_mn_dl, weigth_decay, grad_clip, opt_func)

In [ ]:
NormalCNN_hour_history = []

In [ ]:
NormalCNN_hour_history += NormalCNN_hour.fit(epochs, max_lr, train_hn_dl, test_hn_dl, weigth_decay, grad_clip, opt_func)

RuntimeError: Given groups=1, weight of size [10, 20, 3], expected input[32, 10, 20] to have 20 channels, but got 10 channels instead

### Bidirectional Deep LSTM encoder

In [ ]:
## Architecture
hidden_size = 10
input_size = 20
##nn architecture
architecture = ()

In [ ]:
#encoders
deep_lstm_encoder_min_forward = DeepLSTM(hidden_size, input_size, architecture)
deep_lstm_encoder_min_backward = DeepLSTM(hidden_size, input_size, architecture)
deep_lstm_encoder_min = BidirectionalRNNWithAttention(deep_lstm_encoder_min_forward,deep_lstm_encoder_min_backward)

deep_lstm_encoder_hour_forward = DeepLSTM(hidden_size, input_size, architecture)
deep_lstm_encoder_hour_backward = DeepLSTM(hidden_size, input_size, architecture)
deep_lstm_encoder_hour = BidirectionalRNNWithAttention(deep_lstm_encoder_hour_forward,deep_lstm_encoder_hour_backward)

#fc layer
deep_lstm_fc_dst_min = DeepNeuralNetwork(2*hidden_size, pred_length, *architecture)
deep_lstm_fc_kp_min = DeepNeuralNetwork(2*hidden_size, hour_to_3_hour(pred_length), *architecture)

deep_lstm_fc_dst_hour = DeepNeuralNetwork(2*hidden_size, pred_length, *architecture)
deep_lstm_fc_kp_hour = DeepNeuralNetwork(2*hidden_size, hour_to_3_hour(pred_length), *architecture)

In [ ]:
NormalLSTM_min = to_device(NormalArchitecture(deep_lstm_encoder_min, deep_lstm_fc_dst_min, deep_lstm_fc_kp_min), device)
NormalLSTM_hour = to_device(NormalArchitecture(deep_lstm_encoder_hour, deep_lstm_fc_dst_hour, deep_lstm_fc_kp_hour), device)

In [ ]:
##hyperparameters
epochs = 100
max_lr = 1e-3
weigth_decay = 1e-4
grad_clip = 1e-2
#opt_func = Adam
opt_func = RMSprop

In [ ]:
NormalLSTM_min_history = []

In [ ]:
NormalLSTM_min_history += NormalLSTM_min.fit(epochs, max_lr, train_mn_dl, test_mn_dl, weigth_decay, grad_clip, opt_func)

In [ ]:
NormalLSTM_hour_history = []

In [ ]:
NormalLSTM_hour_history += NormalLSTM_hour.fit(epochs, max_lr, train_hn_dl, test_hn_dl, weigth_decay, grad_clip, opt_func)

Epoch [0]:
	last_lr: 0.00004
	train_loss: 1.3096
	val_loss: 0.9849
Epoch [1]:
	last_lr: 0.00005
	train_loss: 0.9692
	val_loss: 0.8660
Epoch [2]:
	last_lr: 0.00006
	train_loss: 0.8529
	val_loss: 0.7863
Epoch [3]:
	last_lr: 0.00008
	train_loss: 0.7619
	val_loss: 0.7326
Epoch [4]:
	last_lr: 0.00010
	train_loss: 0.7028
	val_loss: 0.6870
Epoch [5]:
	last_lr: 0.00013
	train_loss: 0.6619
	val_loss: 0.6704
Epoch [6]:
	last_lr: 0.00016
	train_loss: 0.6341
	val_loss: 0.6449
Epoch [7]:
	last_lr: 0.00020
	train_loss: 0.6039
	val_loss: 0.6322
Epoch [8]:
	last_lr: 0.00024
	train_loss: 0.5860
	val_loss: 0.6194
Epoch [9]:
	last_lr: 0.00028
	train_loss: 0.5695
	val_loss: 0.5994


KeyboardInterrupt: 

### Bidirectional Deep GRU encoder

In [ ]:
## Architecture
hidden_size = 10
input_size = 20
##nn architecture
architecture = ()

In [ ]:
#encoders

deep_gru_encoder_min_forward = DeepGRU(hidden_size, input_size, architecture)
deep_gru_encoder_min_backward = DeepGRU(hidden_size, input_size, architecture)
deep_gru_encoder_min = BidirectionalRNNWithAttention(deep_gru_encoder_min_forward,deep_gru_encoder_min_backward)

deep_gru_encoder_hour_forward = DeepGRU(hidden_size, input_size, architecture)
deep_gru_encoder_hour_backward = DeepGRU(hidden_size, input_size, architecture)
deep_gru_encoder_hour = BidirectionalRNNWithAttention(deep_gru_encoder_hour_forward,deep_gru_encoder_hour_backward)
#fc layer
deep_gru_fc_dst_min = DeepNeuralNetwork(2*hidden_size, pred_length, *architecture)
deep_gru_fc_kp_min = DeepNeuralNetwork(2*hidden_size,hour_to_3_hour(pred_length), *architecture)

deep_gru_fc_dst_hour = DeepNeuralNetwork(2*hidden_size, pred_length, *architecture)
deep_gru_fc_kp_hour = DeepNeuralNetwork(2*hidden_size, hour_to_3_hour(pred_length), *architecture)

In [ ]:
NormalGRU_min = to_device(NormalArchitecture(deep_gru_encoder_min, deep_gru_fc_dst_min, deep_gru_fc_kp_min), device)
NormalGRU_hour = to_device(NormalArchitecture(deep_gru_encoder_hour, deep_gru_fc_dst_hour, deep_gru_fc_kp_hour), device)

In [ ]:
##hyperparameters
epochs = 100
max_lr = 1e-3
weigth_decay = 1e-4
grad_clip = 1e-2
#opt_func = Adam
opt_func = RMSprop

In [ ]:
NormalGRU_min_history = []

In [ ]:
NormalGRU_min_history += NormalGRU_min.fit(epochs, max_lr, train_mn_dl, test_mn_dl, weigth_decay, grad_clip, opt_func)

RuntimeError: bad allocation

In [ ]:
NormalGRU_hour_history = []

In [ ]:
NormalGRU_hour_history += NormalGRU_hour.fit(epochs, max_lr, train_hn_dl, test_hn_dl, weigth_decay, grad_clip, opt_func)

Epoch [0]:
	last_lr: 0.00004
	train_loss: 0.9408
	val_loss: 0.8459


KeyboardInterrupt: 

### Bidirectional Deep Vanilla RNN encoder

In [ ]:
## Architecture
hidden_size = 10
input_size = 20
##nn architecture
architecture = ()

In [ ]:
#encoders
deep_rnn_encoder_min_forward = DeepVanillaRNN(hidden_size, input_size, architecture)
deep_rnn_encoder_min_backward = DeepVanillaRNN(hidden_size, input_size, architecture)
deep_rnn_encoder_min = BidirectionalRNNWithAttention(deep_rnn_encoder_min_forward, deep_rnn_encoder_min_backward)

deep_rnn_encoder_hour_forward = DeepVanillaRNN(hidden_size, input_size, architecture)
deep_rnn_encoder_hour_backward = DeepVanillaRNN(hidden_size, input_size, architecture)
deep_rnn_encoder_hour = BidirectionalRNNWithAttention(deep_rnn_encoder_hour_forward,deep_rnn_encoder_hour_backward)
#fc layer
deep_rnn_fc_dst_min = DeepNeuralNetwork(2*hidden_size, pred_length, *architecture)
deep_rnn_fc_kp_min = DeepNeuralNetwork(2*hidden_size, hour_to_3_hour(pred_length), *architecture)

deep_rnn_fc_dst_hour = DeepNeuralNetwork(2*hidden_size, pred_length, *architecture)
deep_rnn_fc_kp_hour = DeepNeuralNetwork(2*hidden_size, hour_to_3_hour(pred_length), *architecture)

In [ ]:
NormalVanillaRNN_min = to_device(NormalArchitecture(deep_rnn_encoder_min, deep_rnn_fc_dst_min, deep_rnn_fc_kp_min), device)
NormalVanillaRNN_hour = to_device(NormalArchitecture(deep_rnn_encoder_hour, deep_rnn_fc_dst_hour, deep_rnn_fc_kp_hour), device)

In [ ]:
##hyperparameters
epochs = 100
max_lr = 1e-3
weigth_decay = 1e-4
grad_clip = 1e-2
#opt_func = Adam
opt_func = RMSprop

In [ ]:
NormalVanillaRNN_min_history = []

In [ ]:
NormalVanillaRNN_min_history += NormalVanillaRNN_min.fit(epochs, max_lr, train_mn_dl, test_mn_dl, weigth_decay, grad_clip, opt_func)

In [ ]:
NormalVanillaRNN_hour_history = []

In [ ]:
NormalVanillaRNN_hour_history += NormalVanillaRNN_hour.fit(epochs, max_lr, train_hn_dl, test_hn_dl, weigth_decay, grad_clip, opt_func)

Epoch [0]:
	last_lr: 0.00004
	train_loss: 1.3040
	val_loss: 1.0009
Epoch [1]:
	last_lr: 0.00005
	train_loss: 1.0306
	val_loss: 0.9201
Epoch [2]:
	last_lr: 0.00006
	train_loss: 0.9532
	val_loss: 0.8801


KeyboardInterrupt: 

### Non deep bidirectional architectures with attention

#### LSTM

In [ ]:
non_deep_lstm_forward_min = LSTM(input_size, hidden_size)
non_deep_lstm_backward_min = LSTM(input_size, hidden_size)
bidirectional_lstm_encoder_min = BidirectionalRNNWithAttention(non_deep_lstm_forward_min, non_deep_lstm_backward_min)

non_deep_lstm_forward_hour = LSTM(input_size, hidden_size)
non_deep_lstm_backward_hour = LSTM(input_size, hidden_size)
bidirectional_lstm_encoder_hour = BidirectionalRNNWithAttention(non_deep_lstm_forward_hour, non_deep_lstm_backward_hour)
#fc layer
non_deep_lstm_fc_dst_min = DeepNeuralNetwork(2*hidden_size, pred_length, *architecture)
non_deep_lstm_fc_kp_min = DeepNeuralNetwork(2*hidden_size, hour_to_3_hour(pred_length), *architecture)

non_deep_lstm_fc_dst_hour = DeepNeuralNetwork(2*hidden_size, pred_length, *architecture)
non_deep_lstm_fc_kp_hour = DeepNeuralNetwork(2*hidden_size, hour_to_3_hour(pred_length), *architecture)

In [ ]:
NormalNonDeepLSTM_min = to_device(NormalArchitecture(bidirectional_lstm_encoder_min, non_deep_lstm_fc_dst_min, non_deep_lstm_fc_kp_min), device)
NormalNonDeepLSTM_hour = to_device(NormalArchitecture(bidirectional_lstm_encoder_hour, non_deep_lstm_fc_dst_hour, non_deep_lstm_fc_kp_hour), device)

In [ ]:
NormalNonDeepLSTM_min_history = []

In [ ]:
NormalNonDeepLSTM_min_history += NormalNonDeepLSTM_min.fit(epochs, max_lr, train_mn_dl, test_mn_dl, weigth_decay, grad_clip, opt_func)

In [ ]:
NormalNonDeepLSTM_hour_history = []

In [ ]:
NormalNonDeepLSTM_hour_history += NormalNonDeepLSTM_hour.fit(epochs, max_lr, train_hn_dl, test_hn_dl, weigth_decay, grad_clip, opt_func)

Epoch [0]:
	last_lr: 0.00005
	train_loss: 2.0350
	val_loss: 2.0597
Epoch [1]:
	last_lr: 0.00014
	train_loss: 2.0237
	val_loss: 2.0414
Epoch [2]:
	last_lr: 0.00026
	train_loss: 1.9437
	val_loss: 1.7768


KeyboardInterrupt: 

#### GRU

In [ ]:
non_deep_gru_forward_min = GRU(input_size, hidden_size)
non_deep_gru_backward_min = GRU(input_size, hidden_size)
bidirectional_gru_encoder_min = BidirectionalRNNWithAttention(non_deep_gru_forward_min, non_deep_gru_backward_min)

non_deep_gru_forward_hour = GRU(input_size, hidden_size)
non_deep_gru_backward_hour = GRU(input_size, hidden_size)
bidirectional_gru_encoder_hour = BidirectionalRNNWithAttention(non_deep_gru_forward_hour, non_deep_gru_backward_hour)
#fc layer
non_deep_gru_fc_dst_min = DeepNeuralNetwork(2*hidden_size, pred_length, *architecture)
non_deep_gru_fc_kp_min = DeepNeuralNetwork(2*hidden_size, hour_to_3_hour(pred_length), *architecture)

non_deep_gru_fc_dst_hour = DeepNeuralNetwork(2*hidden_size, pred_length, *architecture)
non_deep_gru_fc_kp_hour = DeepNeuralNetwork(2*hidden_size, hour_to_3_hour(pred_length), *architecture)


In [ ]:
NormalNonDeepGRU_min = to_device(NormalArchitecture(bidirectional_gru_encoder_min, non_deep_gru_fc_dst_min, non_deep_gru_fc_kp_min), device)
NormalNonDeepGRU_hour = to_device(NormalArchitecture(bidirectional_gru_encoder_hour, non_deep_gru_fc_dst_hour, non_deep_gru_fc_kp_hour), device)

In [ ]:
NormalNonDeepGRU_min_history = []

In [ ]:
NormalNonDeepGRU_min_history += NormalNonDeepGRU_min.fit(epochs, max_lr, train_mn_dl, test_mn_dl, weigth_decay, grad_clip, opt_func)

In [ ]:
NormalNonDeepGRU_hour_history = []

In [ ]:
NormalNonDeepGRU_hour_history += NormalNonDeepGRU_hour.fit(epochs, max_lr, train_hn_dl, test_hn_dl, weigth_decay, grad_clip, opt_func)

Epoch [0]:
	last_lr: 0.00005
	train_loss: 2.1161
	val_loss: 2.1215
Epoch [1]:
	last_lr: 0.00014
	train_loss: 2.0831
	val_loss: 2.0747


KeyboardInterrupt: 

#### RNN

In [ ]:
non_deep_rnn_forward_min = VanillaRNN(input_size, hidden_size)
non_deep_rnn_backward_min = VanillaRNN(input_size, hidden_size)
bidirectional_rnn_encoder_min = BidirectionalRNNWithAttention(non_deep_rnn_forward_min, non_deep_rnn_backward_min)

non_deep_rnn_forward_hour = VanillaRNN(input_size, hidden_size)
non_deep_rnn_backward_hour = VanillaRNN(input_size, hidden_size)
bidirectional_rnn_encoder_hour = BidirectionalRNNWithAttention(non_deep_rnn_forward_hour, non_deep_rnn_backward_hour)
#fc layer
non_deep_rnn_fc_dst_min = DeepNeuralNetwork(2*hidden_size, pred_length, *architecture)
non_deep_rnn_fc_kp_min = DeepNeuralNetwork(2*hidden_size, hour_to_3_hour(pred_length), *architecture)

non_deep_rnn_fc_dst_hour = DeepNeuralNetwork(2*hidden_size, pred_length, *architecture)
non_deep_rnn_fc_kp_hour = DeepNeuralNetwork(2*hidden_size, hour_to_3_hour(pred_length), *architecture)


In [ ]:
NormalNonDeepVanillaRNN_min = to_device(NormalArchitecture(bidirectional_rnn_encoder_min, non_deep_rnn_fc_dst_min, non_deep_rnn_fc_kp_min), device)
NormalNonDeepVanillaRNN_hour = to_device(NormalArchitecture(bidirectional_rnn_encoder_hour, non_deep_rnn_fc_dst_hour, non_deep_rnn_fc_kp_hour), device)

In [ ]:
NormalNonDeepVanillaRNN_min_history = []

In [ ]:
RefinedNonDeepVanillaRNN_min_history += NormalNonDeepVanillaRNN_min.fit(epochs, max_lr, train_mn_dl, test_mn_dl, weigth_decay, grad_clip, opt_func)

In [ ]:
NormalNonDeepVanillaRNN_hour_history = []

In [ ]:
NormalNonDeepVanillaRNN_hour_history += NormalNonDeepVanillaRNN_hour.fit(epochs, max_lr, train_hn_dl, test_hn_dl, weigth_decay, grad_clip, opt_func)

Epoch [0]:
	last_lr: 0.00005
	train_loss: 2.0527
	val_loss: 2.0624


KeyboardInterrupt: 

# Conclusion